In [1]:
import pandas 
import psycopg2
import datetime
import time
import os
conn = psycopg2.connect(host = '127.0.0.1' , dbname = 'postgres' , user = 'postgres' , password = '')
cur = conn.cursor()
pandas.set_option('display.max_rows',5000)

# 【 Process Date Range 】

In [2]:
def PROCESS_DATE_RANGE(start = datetime.datetime.now() , end = datetime.datetime.now()):
    
    da = pandas.date_range(start = start ,end = end)
    work_D = []
    
    for i in da :
        if i.weekday() in range(0,5):
            work_D.append(i)
    
    return work_D

# 【 Stock 】- Daily

In [3]:
def GET_STOCK_DAILY(work_D,cursor):
    
    cur = cursor
    list_url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=html&date={:0>4}{:0>2}{:0>2}&type=ALLBUT0999'
    list_sql = 'insert into stock_daily (no,name,close,fluctuation,open,highest,lowest,quantity,date) \
                values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    
    for D in work_D:
     
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day) 
        
        try:
            stock = pandas.read_html(list_url.format(D.year , D.month , D.day))[8]
            stock = stock.iloc[:,[0,1,8,9,10,5,6,7,2]]
            stock.columns = ['no','name','close','fluc(+/-)','fluc','open','highest','lowest','quantity']
            stock['store'] = stock.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            stock = stock[stock['store'] == 'y']
            stock['date'] = process_day
            stock['quantity'] = round(stock['quantity']/1000,0)
            stock['close'] = stock['close'].str.replace('--','0')
            stock['open'] = stock['open'].str.replace('--','0')
            stock['highest'] = stock['highest'].str.replace('--','0')
            stock['lowest'] = stock['lowest'].str.replace('--','0')
            stock['fluctuation'] = stock.apply(lambda x : x['fluc']*(-1) if x['fluc(+/-)'] == '-' else x['fluc'] , axis = 1)
            stock = stock.iloc[:,[0,1,2,4,5,6,7,8,10]]
            
            for _,data in stock.iterrows():
                cur.execute(list_sql,data)
            cur.execute('commit')           
            
            print('【 Stock Daily Data 】{} data inserted .'.format(process_day))
            stock = ''
            time.sleep(20)
            
        except ValueError:
            print('【 Stock Daily Data 】{} No data'.format(process_day))
            pass

# 【 Stock 】 - Daily Average

In [4]:
def GET_STOCK_DAILY_AVG(work_D,cursor):
    
    cur = cursor
    list_url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=html&date={:0>4}{:0>2}{:0>2}&type=ALLBUT0999'
    avg_p_sql = 'insert into stock_daily_avg_price (date , no , price) values (%s,%s,%s)'
    
    for D in work_D:
        

        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)
        
        try:        
            avg_stock = pandas.read_html(list_url.format(D.year,D.month,D.day))[8]
            avg_stock = avg_stock.iloc[:,[0,2,4]]
            avg_stock.columns = ['no','total_q','total_a']
            avg_stock['store'] = avg_stock.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            avg_stock = avg_stock[avg_stock['store']=='y']
            avg_stock['avg_price'] = round(avg_stock['total_a']/avg_stock['total_q'],2)
            avg_stock['date'] = process_day
            avg_stock = avg_stock[['date','no','avg_price']]
            
            for _,data in avg_stock.iterrows():
                cur.execute(avg_p_sql,data)
            cur.execute('commit')

            print('【 Stock Average price 】{} data inserted .'.format(process_day))
            time.sleep(20)
            
        except ValueError :
            print('【 Stock Average price 】{} No data'.format(process_day))
            pass
    

In [ ]:
work_D = PROCESS_DATE_RANGE()
work_D
STOCK_FI_OBS(work_D,cur)

【 Foreign Investor (Stock) Over Bought / Sold 】2021-07-26 data inserted .


# 【 Stock 】- Foreign Investor Buy / Sell

In [3]:
def STOCK_FI_OBS(work_D,cursor):
    
    cur = cursor
    foreign_investor_stcok_url = 'https://www.twse.com.tw/fund/TWT38U?response=html&date={:0>4}{:0>2}{:0>2}'   
    foreign_investor_sql = 'insert into foreign_investor (date , no , quantity) values (%s,%s,%s)'
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)     
    
        try:
            fi = pandas.read_html(foreign_investor_stcok_url.format(D.year,D.month,D.day))[0]
            fi = fi.iloc[:,[1,11]]
            fi.columns = ['no','quantity']
            fi['no'] = fi['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            fi['store'] = fi.apply(lambda x : 'y' if len(x.loc['no']) == 4 else 'n' ,axis =1)
            fi = fi[fi['store']=='y']
            fi['date'] = process_day
            fi = fi.iloc[:,[3,0,1]]
            
            for _ , data in fi.iterrows():               
                cur.execute(foreign_investor_sql,data)
                cur.execute('commit')
            cur.execute('commit')
            
            
            print('【 Foreign Investor (Stock) Over Bought / Sold 】{} data inserted .'.format(process_day))
            time.sleep(20)
            
        except ValueError:
            print(' Foreign Investor (Stock) Over Bought / Sold 】{} No data'.format(process_day))
            pass

# 【 Stock 】 - Investment Trust Buy / Sell

In [6]:
def STOCK_IT_OBS(work_D,cursor):
    
    cur = cursor
    investment_trust_stock_url ='https://www.twse.com.tw/fund/TWT44U?response=html&date={:0>4}{:0>2}{:0>2}'
    investment_trust_sql = 'insert into investment_trust (date , no , quantity) values (%s,%s,%s)'
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day) 
        
    # Investment Trust (Stock) Over Bought/Sold    
        
        try:
            process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
            investment_trust = pandas.read_html(investment_trust_stock_url.format(D.year,D.month,D.day))[0].iloc[:,[1,5]]
            investment_trust.columns = ['no','quantity']
            investment_trust['no'] = investment_trust['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            investment_trust['store'] = investment_trust.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            investment_trust= investment_trust[investment_trust['store']=='y']
            investment_trust['date'] = process_day
            investment_trust = investment_trust.iloc[:,[3,0,1]]
            
            for _ , data in investment_trust.iterrows():
                cur.execute(investment_trust_sql,data)
            cur.execute('commit')
            
            print('【 Investment Trust (Stock) Over Bought / Sold 】{} data inserted .'.format(process_day))
            time.sleep(20)
            
        except ValueError:
            print('【 Investment Trust (Stock) Over Bought / Sold 】{} No data'.format(process_day))
            pass
        

# 【 Stock 】- Dealer Buy / Sell

In [7]:
def STOCK_DL_OBS(work_D,cursor):
    
    cur = cursor
    dealer_stock_url ='https://www.twse.com.tw/fund/TWT43U?response=html&date={:0>4}{:0>2}{:0>2}'
    dealer_sql = 'insert into dealer (date , no , quantity) values (%s,%s,%s)'
    
    for D in work_D:    
    
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)   
    
        try:
            dealer = pandas.read_html(dealer_stock_url.format(D.year,D.month,D.day))[0]
            dealer = dealer.iloc[:,[0,10]]
            dealer.columns = ['no','quantity']
            dealer['no'] = dealer['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            dealer['store'] = dealer.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis=1)
            dealer = dealer[dealer['store']=='y']
            dealer = dealer[dealer['quantity'] != 0]
            dealer['date'] = process_day
            dealer = dealer.iloc[:,[3,0,1]]
            
            for _ , data in dealer.iterrows():
                cur.execute(dealer_sql,data)
            cur.execute('commit')

            print('【 Dealer (Stock) Over Bought / Sold 】{} data inserted .'.format(process_day))
            time.sleep(20)
            
        except ValueError:
            print('【 Dealer (Stock) Over Bought / Sold 】{} No data'.format(process_day))   
            pass

# 【 Stock 】 - Credit

In [ ]:
def STOCK_CREDIT(work_D,cur):
    for D in work_D:
        insert_D = D
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(insert_D.year,insert_D.month,insert_D.day)
        try :
            main = pandas.read_html("https://www.twse.com.tw/exchangeReport/MI_MARGN?response=html&date={:0>4}{:0>2}{:0>2}&selectType=ALL".format(insert_D.year,insert_D.month,insert_D.day))[1].iloc[:,[0,6,12,13]]
            main.columns = ['no','margin','short','total']
            main['date'] = process_day
            main['store'] = main.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            main = main[main['store']=='y']
            main = main[['date','no','margin','short','total']]
            
            for _ , data in main.iterrows():
                cur.execute("insert into credit_trade (date,no,margin,short,total) values (%s,%s,%s,%s,%s)" , data)
            cur.execute("commit")
            print('【 (STOCK) Credit Trade 】{} data inserted.'.format(process_day))
            time.sleep(20)
                    
        except ValueError :
            print('【 (STOCK) Credit Trade 】{} No data'.format(process_day))
            pass

# 【 Stock 】 - Shareholding

In [ ]:
def STOCK_FI_SHAREHOLDING(work_D,cur):
    for D in work_D:
        insert_D = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
        html = 'https://www.twse.com.tw/fund/MI_QFIIS?response=html&date={:0>4}{:0>2}{:0>2}&selectType=ALLBUT0999'.format(D.year,D.month,D.day)
        try:
            main = pandas.read_html(html)[0]
            main = main.iloc[:,[0,7,3]]
            main.columns = ['no','hold_percent','issued_amount']
            main['date'] = insert_D
            main['hold_percent'] = main['hold_percent']/100
            main['store'] = main.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis =1 )
            main = main[main['store']=='y']
            main = main[['date','no','hold_percent','issued_amount']]
            
            for _ , data in main.iterrows():
                cur.execute("insert into FI_HOLD (date , no , hold_percent,issued_amount) values (%s,%s,%s,%s)",data)
            cur.execute("commit")
            print("【 Stock Share Holding 】{} data inserted. ".format(insert_D))
            
            time.sleep(15)    
            
        except ValueError :
            print("【 Stock Share Holding 】{} no data. ".format(insert_D))

# 【 Stock 】 - Issued Amounts

In [5]:
def STOCK_ISSUED_AMOUNTS(work_D , cur):
    for D in work_D:
        insert_D = '{:0>4}{:0>2}{:0>2}'.format(D.year,D.month,D.day)
        html = 'https://www.twse.com.tw/fund/MI_QFIIS?response=html&date={0}&selectType=ALLBUT0999'.format(insert_D)
        try:
            main = pandas.read_html(html)[0]
            main = main.iloc[:,[0,3]]
            main['date'] = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
            main.columns = ['no','amounts','date']
            main['store'] = main.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            main['type'] = 'STOCK'
            main = main[main['store'] == 'y'][['date','no','amounts','type']]
            
            for _ , data in main.iterrows():
                cur.execute("insert into issued_amounts (date , no , amounts,type) values (%s,%s,%s,%s)",data)
            cur.execute("commit")
            
            print('【 Stock issued amounts 】{} data inserted . '.format(insert_D))
            time.sleep(10)
            
        except ValueError:
            print('【 Stock issued amounts 】{} No data'.format(insert_D))
            pass


# 【 OTC 】 - Daily

In [8]:
def GET_OTC_DAILY(work_D,cursor):
    
    cur = cursor
    otc_url = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&o=htm&d={:0>3}/{:0>2}/{:0>2}&s=0,asc,0'
    otc_sql = 'insert into otc_daily (no,name,close,fluctuation,open,highest,lowest,quantity,date) \
               values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)            
        process_otc = pandas.read_html(otc_url.format(D.year-1911 , D.month , D.day))[0]
        
        if process_otc.iloc[-1,0] != '共0筆':
            
            process_otc = process_otc.iloc[:-4,[0,1,2,3,4,5,6,8]]
            process_otc.columns = ['no','name','close','fluctuation','open','highest','lowest','quantity']
            process_otc = process_otc[process_otc['no'] != '管理股票']
            process_otc['quantity'] = round(process_otc['quantity'].astype(int)/1000,0).astype(int).astype(int)
            process_otc['close'] = process_otc['close'].str.replace('---','0')
            process_otc['open'] = process_otc['open'].str.replace('---','0')
            process_otc['highest'] = process_otc['highest'].str.replace('---','0')
            process_otc['lowest'] = process_otc['lowest'].str.replace('---','0')
            process_otc['date'] = process_day
            process_otc['store'] = ''
            process_otc['store'] = process_otc.apply(lambda x : 'Y' if len(x['no']) == 4 else 'N' , axis =1)
            process_otc = process_otc[process_otc['store'] == 'Y'].iloc[:,:-1]
            
            for _,row_data in process_otc.iterrows():
                cur.execute(otc_sql,row_data)
            cur.execute('commit')

            print('【 OTC Daily Data 】{} data inserted .'.format(process_day))
            process_otc =''
            time.sleep(10)
        else:
            print('【 OTC Daily Data 】{} No data '.format(process_day))
        


# 【 OTC 】- Daily Average

In [9]:
def GET_OTC_DAILY_AVG(work_D,cursor):
    
    otc_url = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&o=htm&d={:0>3}/{:0>2}/{:0>2}&s=0,asc,0'
    avg_p_sql = 'insert into stock_daily_avg_price (date , no , price) values (%s,%s,%s)'
    
    for D in work_D:        
                
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)
        process_otc = pandas.read_html(otc_url.format(D.year-1911 , D.month , D.day))[0]
        process_otc = process_otc.iloc[:-4,[0,7]]
        process_otc.columns = ['no','price']
        process_otc['date'] = process_day
        process_otc['store']=''
        process_otc['store'] = process_otc.apply(lambda x : 'n' if len(x['no']) != 4 or x['no'] == '管理股票' else 'y' , axis = 1)
        process_otc = process_otc[process_otc['store'] == 'y']
        process_otc= process_otc.iloc[:,[2,0,1]]
        
        for _,data in process_otc.iterrows():
            cur.execute(avg_p_sql , data)
        cur.execute('commit')        

        print('【 OTC Average price 】{} data inserted .'.format(process_day))
        time.sleep(10)
 

# 【 OTC 】- Foreign Investor Buy / Sell

In [10]:
def OTC_FI_OBS(work_D,cursor):
    
    cur = cursor
    foreign_investor_otc_buy_url = 'https://www.tpex.org.tw/web/stock/3insti/qfii_trading/forgtr_result.php?l=zh-tw&t=D&type=buy&d={:}/{:0>2}/{:0>2}&s=0,asc&o=htm'
    foreign_investor_otc_sell_url = 'https://www.tpex.org.tw/web/stock/3insti/qfii_trading/forgtr_result.php?l=zh-tw&t=D&type=sell&d={:}/{:0>2}/{:0>2}&s=0,asc&o=htm'
    foreign_investor_sql = 'insert into foreign_investor (date , no , quantity) values (%s,%s,%s)'

    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)   

# Foreign investor (OTC) Over Bought

        fi_otc = pandas.read_html(foreign_investor_otc_buy_url.format(D.year-1911,D.month,D.day))[0]
        if fi_otc.iloc[-1,0] != '共0筆':
            fi_otc = fi_otc.iloc[:,[1,11]]
            fi_otc.columns = ['no','quantity']
            fi_otc['no'] = fi_otc['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            fi_otc['store'] = fi_otc.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' ,axis =1 )
            fi_otc = fi_otc[fi_otc['store']=='y']
            fi_otc['quantity'] = fi_otc['quantity'].astype('int')*1000
            fi_otc['date'] = process_day
            fi_otc = fi_otc.iloc[:,[3,0,1]]
            
            for _ , data in fi_otc.iterrows():
                cur.execute(foreign_investor_sql,data)
            cur.execute('commit')
            
            print('【 Foreign Investor (otc) Over Bought 】{} data inserted .'.format(process_day))
        else :
            print('【 Foreign Investor (otc) Over Bought 】{} No data .'.format(process_day))
    
# Foreign investor (OTC) Over Sold       

        fi_otc = pandas.read_html(foreign_investor_otc_sell_url.format(D.year-1911,D.month,D.day))[0]
        if fi_otc.iloc[-1,0] != '共0筆':
            fi_otc = fi_otc.iloc[:,[1,11]]
            fi_otc.columns = ['no','quantity']
            fi_otc['no'] = fi_otc['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            fi_otc['store'] = fi_otc.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' ,axis =1 )
            fi_otc = fi_otc[fi_otc['store']=='y']
            fi_otc['quantity'] = fi_otc['quantity'].astype('int')*1000
            fi_otc['date'] = process_day
            fi_otc = fi_otc.iloc[:,[3,0,1]]
            
            for _ , data in fi_otc.iterrows():
                cur.execute(foreign_investor_sql,data)
            cur.execute('commit')
            
            print('【 Foreign Investor (otc) Over Sold 】{} data inserted.'.format(process_day))
            time.sleep(10)
        else:
            print('【 Foreign Investor (otc) Over Sold 】{} No data .'.format(process_day))


# 【 OTC 】- Investment Trust Buy / Sell

In [11]:
def OTC_IT_OBS(work_D,cursor):
    
    cur = cursor
    investment_trust_sql = 'insert into investment_trust (date , no , quantity) values (%s,%s,%s)'
    investment_trust_otc_sell_url = 'https://www.tpex.org.tw/web/stock/3insti/sitc_trading/sitctr_result.php?l=zh-tw&t=D&type=sell&d={:0>3}/{:0>2}/{:0>2}&o=htm'
    investment_trust_otc_buy_url = 'https://www.tpex.org.tw/web/stock/3insti/sitc_trading/sitctr_result.php?l=zh-tw&t=D&type=buy&d={:0>3}/{:0>2}/{:0>2}&o=htm'

    for D in work_D:
    
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)    
        
# Investment Trust (OTC) Over Bought

        it = pandas.read_html(investment_trust_otc_buy_url.format(D.year-1911,D.month,D.day))[0]
        if it.iloc[-1,0] != '共0筆':
            it = it.iloc[:-1,[1,5]]
            it.columns = ['no','quantity']
            it['no'] = it['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            it['quantity'] = it['quantity'].astype(int)*1000
            it['store'] = ''
            it['store'] = it.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            it = it[it['store'] == 'y']
            it['date'] = process_day
            it = it.iloc[:,[3,0,1]]

            for _ , data in it.iterrows():
                cur.execute(investment_trust_sql,data)
            cur.execute('commit')

            print('【 Investment Trust (Otc) Over Bought 】{} data inserted.'.format(process_day))
        else :
            print('【 Investment Trust (Otc) Over Bought 】{} No data .'.format(process_day))

# Investment True (OTC) Over Sold

        it = pandas.read_html(investment_trust_otc_sell_url.format(D.year-1911,D.month,D.day))[0]
        if it.iloc[-1,0] != '共0筆':
            it = it.iloc[:-1,[1,5]]
            it.columns = ['no','quantity']
            it['no'] = it['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            it['quantity'] = it['quantity'].astype(int)*1000
            it['store'] = ''
            it['store'] = it.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            it = it[it['store'] == 'y']
            it['date'] = process_day
            it = it.iloc[:,[3,0,1]]
            
            for _ , data in it.iterrows():
                cur.execute(investment_trust_sql,data)
            cur.execute('commit')
            
            print('【 Investment Trust (Otc) Over Sold 】{} data inserted.'.format(process_day))
            time.sleep(10)
        else :
            print('【 Investment Trust (Otc) Over Sold 】{} No data .'.format(process_day))

# 【 OTC 】 - Dealer Buy / Sell

In [12]:
def OTC_DL_OBS(work_D,cursor):
    
    cur = cursor
    dealer_otc_buy_url = 'https://www.tpex.org.tw/web/stock/3insti/dealer_trading/dealtr_hedge_result.php?l=zh-tw&o=htm&t=D&type=buy&d={:0>3}/{:0>2}/{:0>2}&s=0,asc'
    dealer_otc_sell_url = 'https://www.tpex.org.tw/web/stock/3insti/dealer_trading/dealtr_hedge_result.php?l=zh-tw&o=htm&t=D&type=sell&d={:0>3}/{:0>2}/{:0>2}&s=0,asc'
    dealer_sql = 'insert into dealer (date , no , quantity) values (%s,%s,%s)'
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)    
    
# Dealer (OTC) Over Bought

        dealer = pandas.read_html(dealer_otc_buy_url.format(D.year-1911,D.month,D.day))[0]
        if dealer.iloc[-1,0] != '共0筆':
            dealer = dealer.iloc[:,[1,9]]
            dealer.columns = ['no','quantity']
            dealer['no'] = dealer['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            dealer['store'] = ''
            dealer['store'] = dealer.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            dealer = dealer[dealer['store'] == 'y']
            dealer['quantity'] = dealer['quantity'].astype(int)*1000
            dealer['date'] = process_day
            dealer = dealer.iloc[:,[3,0,1]]
            
            for _ , data in dealer.iterrows():
                cur.execute(dealer_sql,data)
            cur.execute('commit')
            
            print('【 Dealer (Otc) Over Bought 】{} data inserted.'.format(process_day))
        else:
            print('【 Dealer (Otc) Over Bought 】{} No data inserted.'.format(process_day))

    
# Dealer (OTC) Over Sold    
    
        dealer = pandas.read_html(dealer_otc_sell_url.format(D.year-1911,D.month,D.day))[0]
        if dealer.iloc[-1,0] != '共0筆':
            dealer = dealer.iloc[:,[1,9]]
            dealer.columns = ['no','quantity']
            dealer['no'] = dealer['no'].astype('string').str.pad(width = 4 , side = 'left' , fillchar = '0')
            dealer['store'] = ''
            dealer['store'] = dealer.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            dealer = dealer[dealer['store'] == 'y']
            dealer['quantity'] = dealer['quantity'].astype(int)*1000
            dealer['date'] = process_day
            dealer = dealer.iloc[:,[3,0,1]]
            
            for _ , data in dealer.iterrows():
                cur.execute(dealer_sql,data)
            cur.execute('commit')            

            print('【 Dealer (Otc) Over Sold 】{} data inserted.'.format(process_day))
            time.sleep(10)
        else:
            print('【 Dealer (Otc) Over Sold 】{} No data .'.format(process_day))

# 【 OTC 】 - Credit

In [3]:
def OTC_CREDIT(work_D,cur):
    for D in work_D:
        insert_D = D
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(insert_D.year,insert_D.month,insert_D.day)
        
        try:
            main = pandas.read_html("https://www.tpex.org.tw/web/stock/margin_trading/margin_balance/margin_bal_result.php?l=zh-tw&o=htm&d={0}/{1:0>2}/{2:0>2}&s=0,asc".format(insert_D.year-1911,insert_D.month,insert_D.day))[0].iloc[:-4,[0,6,14,17]]
            main.columns = ['no','margin','short','total']
            main['date'] = process_day
            main['store'] = main.apply(lambda x : 'y' if len(x['no']) == 4 else 'n' , axis = 1)
            main = main[main['store']=='y']
            main = main[['date','no','margin','short','total']]
            
            
            for _ , data in main.iterrows():
                cur.execute("insert into credit_trade (date,no,margin,short,total) values (%s,%s,%s,%s,%s)" , data)
            cur.execute("commit")
            print('【 (OTC) Credit Trade 】{} data inserted.'.format(process_day))
            time.sleep(10)
                    
        except ValueError :
            print('【 (OTC) Credit Trade 】{} No data'.format(process_day))
            pass

# 【 OTC 】- Issued Amounts

In [ ]:
def OTC_ISSUED_AMOUNTS(work_D,cur):
    for D in work_D:
        insert_D = '{:0>4}{:0>2}{:0>2}'.format(D.year,D.month,D.day)
        html = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&o=htm&d={:0>3}/{:0>2}/{:0>2}&s=0,asc,0'.format(D.year-1911,D.month,D.day)
        try:
            main = pandas.read_html(html)[0].iloc[:,[0,15]]
            main['not_stock']= main.apply(lambda x : 'y' if x.iloc[0].isdigit() == True else 'n' , axis =1)
            main = main[main['not_stock'] == 'y'].iloc[:,[0,1]]
            main['not_etl']= main.apply(lambda x : 'y' if len(x.iloc[0]) == 4  else 'n' , axis =1 )
            main = main[main['not_etl'] == 'y'].iloc[:,[0,1]]
            main.columns = ['no','issued_amounts']
            #main = main[main['no'] != '8462']
            main['date'] = insert_D
            main['type'] = 'OTC'
        
            for _ , data in main.iterrows():
                cur.execute("insert into issued_amounts (no , amounts , date , type) values (%s,%s,%s,%s)",data)
            cur.execute("commit")
            
            print('【 OTC issued amounts 】{} data inserted . '.format(insert_D))
            time.sleep(10)
        
        except ValueError:
            print('【 OTC issued amounts 】{} No data'.format(insert_D))
            pass

# 【 Main Corporation 】

In [13]:
def MAIN_CORP(work_D,cursor):
    
    cur = cursor
    corp_url = 'http://jsjustweb.jihsun.com.tw/z/zg/zgb/zgb0.djhtm?a={0}&b={1}&c=B&e={2}&f={2}'
    zco_sql = 'insert into zco (stock_number , buy_amount , sell_amount , corp ,date) values (%s,%s,%s,%s,%s)'
    zco_dict = pandas.read_csv('./branch_code.csv')
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)
        
        for idx in range(len(zco_dict)):
            zco_corp , zco_mc , zco_bc  = zco_dict['Corp.'][idx],zco_dict['Master code'][idx] , zco_dict['Branch code'][idx]
            zco = pandas.read_html('http://jsjustweb.jihsun.com.tw/z/zg/zgb/zgb0.djhtm?a={0}&b={1}&c=B&e={2}&f={2}'.format(zco_mc,zco_bc,process_day))
            zco_b , zco_s = zco[3],zco[4]
            zco_s['corp'] ,zco_b['corp']= zco_corp,zco_corp
            zco_s[0] = zco_s[0].str.extract(r'AS(\d{4})')
            zco_b[0] = zco_b[0].str.extract(r'AS(\d{4})')
            zco = pandas.concat([zco_s,zco_b])
            zco['date'] = process_day
            zco = zco.iloc[:,[0,1,2,4,5]]
            zco.columns = ['NO','buy amount','sell amount','corp','date']
            zco = zco[zco['NO'].notnull()]
            
            for _ , data in zco.iterrows():
                cur.execute(zco_sql,data)
            cur.execute('commit')
             
        print('【 Main Corp. Transaction Detail 】{} data inserted .'.format(process_day))
        zco,zco_b,zco_s = '','',''

# 【 Work Date 】

In [5]:
def WORK_DATE(work_D,cursor):
# Truncate table work_date and reinsert date data
    for D in work_D:
        
        insert_D = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)        
        query_D = '{:0>3}/{:0>2}/{:0>2}'.format(D.year-1911,D.month,D.day)
        try:
            if pandas.read_html("https://www.tpex.org.tw/web/stock/aftertrading/index_summary/summary_print.php?l=zh-tw&d={}&s=0,asc,0".format(query_D))[0].iloc[2,0] != '':
                if pandas.read_sql("select * from work_date where date = '{}'".format(insert_D), con = conn).shape[0] == 0:
                    cur.execute("insert into work_date (date) values ('{}')".format(insert_D))
                    cur.execute('commit')
                    print(f'{insert_D} insert to work_date.')
                    time.sleep(3)
                else:
                    print('{} is exist.'.format(insert_D))
            else :
                print(f'{insert_D} failed.')
        except IndexError:
            print('{} No data'.format(insert_D))
            pass